In [13]:
from google.colab import drive
import pandas as pd
import sqlite3
import os

drive.mount('/content/gdrive')

caminho_banco = '/content/gdrive/MyDrive/SQL/'
nomedodataset = 'dataset_bancomundial (1).csv'

if not os.path.exists(caminho_banco):
    os.makedirs(caminho_banco)
    print("Pasta 'SQL' criada com sucesso!")

caminho_arquivo_csv = '/content/gdrive/MyDrive/' + nomedodataset
dados = pd.read_csv(caminho_arquivo_csv, sep=';', decimal=',')
caminho_banco_dados = caminho_banco + 'dataset_bancomundial.db'

def table_exists(conn, table_name):
    cursor = conn.cursor()
    cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}'")
    return cursor.fetchone() is not None

def criar_tabelas(conn):
    comandos_sql = [
        """
        CREATE TABLE IF NOT EXISTS Country (
            CountryCode VARCHAR PRIMARY KEY NOT NULL,
            ShortName VARCHAR NOT NULL,
            LongName VARCHAR NOT NULL,
            Region VARCHAR NOT NULL
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS Indicator (
            IndicatorCode VARCHAR PRIMARY KEY NOT NULL,
            IndicatorName VARCHAR NOT NULL
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS Data (
            CountryCode VARCHAR,
            IndicatorCode VARCHAR,
            Year INTEGER,
            Value INTEGER,
            FOREIGN KEY(CountryCode) REFERENCES Country(CountryCode),
            FOREIGN KEY(IndicatorCode) REFERENCES Indicator(IndicatorCode)
        );
        """
    ]

    for comando in comandos_sql:
        conn.execute(comando)

def salvar_dados_no_banco(dados, nome_tabela, caminho_banco_dados):
    if os.path.exists(caminho_banco_dados):
        os.remove(caminho_banco_dados)

    conn = sqlite3.connect(caminho_banco_dados)
    if not table_exists(conn, nome_tabela):
        criar_tabelas(conn)
        dados.to_sql(nome_tabela, conn, index=False)
        print(f"Tabela '{nome_tabela}' foi criada e os dados foram salvos com sucesso no banco de dados.")
    else:
        print(f"A tabela '{nome_tabela}' já existe no banco de dados.")

    conn.close()

nome_tabela = 'dados_bancomundial'
salvar_dados_no_banco(dados, nome_tabela, caminho_banco_dados)

with sqlite3.connect(caminho_banco_dados) as conn:
    tabelas = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", conn)
    display(tabelas)

    cursor = conn.cursor()

    sql_command = "SELECT * FROM dados_bancomundial;"
    cursor.execute(sql_command)

    result = cursor.fetchall()

    for row in result:
        print(row)


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Pasta 'SQL' criada com sucesso!
Tabela 'dados_bancomundial' foi criada e os dados foram salvos com sucesso no banco de dados.


,name
0,Country
1,Indicator
2,Data
3,dados_bancomundial


Streaming output truncated to the last 5000 lines.
(270, 'Algeria', 'DZA', 'Domestic credit to private sector (% of GDP)', 'GFDD.DI.14', None, None, None, None, 14.94764137, 18.10898781, 15.54507637, 17.23931885, 23.5247345, 32.18621445, 28.4405098, 35.00871277, 47.72993851, 49.13802338, 42.00649643, 49.96541214, 53.55384827, 49.90439987, 53.82227707, 51.98721695, 48.50830841, 52.57105637, 62.16136932, 65.06717682, 66.49738312, 68.72982788, 69.3118515, 67.53259277, 68.14678192, 63.19371414, 56.14321518, 46.2891655, 7.254817486, 6.617753029, 6.489104748, 5.199387074, 5.364902496, 3.907416821, 4.563822269, 5.388085842, 5.965660095, 8.06467247, 12.29866982, 11.38743114, 11.11569023, 12.01138973, 12.35186481, 13.43868065, 13.16554832, 16.61234474, 15.63307476, 14.00934029, 14.54335976, 16.69387436)
(271, 'Algeria', 'DZA', 'Electronic payments used to make payments (% age 15+)', 'GFDD.AI.22', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, Non

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

